# 7.1 Sequential 모델을 넘어서: 케라스의 함수형 API

Sequential 모델은 네트워크의 입력과 출력이 하나로 가정한 뒤 층을 차례대로 쌓아 구성.

하지만 이런 가정이 맞지 않는 경우도 많음. 일부 네트워크는 개별 입력이 여러 개 필요하거나 출력이 여러 개 필요하며, 층을 차례대로 쌓지 않고 층 사이를 연결하여 그래프처럼 만드는 네트워크도 존재.

인풋 데이터로 여러 타입의 데이터를 입력받을 때(연도 브랜드 등 - 완전연결모듈, 텍스트 - RNN or Conv1D, 이미지 - Conv2D) 각각의 모델을 따로 훈련하고 각 예측의 가중평균을 통해 구할 수 있지만, 이는 최적의 방법이 아님

가능한 모든 종류의 입력 데이터를 동시에 사용해서 정확한 하나의 모델을 학습하는 것이 더 나은 방법.

위의 경우 완전연결모듈, RNN 모듈, 컨브넷 모듈로 나누어 3개의 입력가지를 통해 하나의 모델을 학습하는 경우가 베스트

또는 입력데이터로부터 여러 개의 타깃 속성을 예측해야 하는 경우도 존재

짧은 글을 입력받았을 때 장르와 글이 써진 대략의 시대를 예측해야하는 경우가 하나의 예.

이 경우도 마찬가지로 2개의 모델을 따로 훈련할 수 있지만 이 속성들은 통계적으로 독립적이지 않기 때문에 동시에 같이 예측하도록 해야 더 좋은 모델이 나옴

기존에는 여러 모델을 거쳐 하나의 텐서로 합쳐지는 모델이 전통적이였으나, 최근에는 하위층의 텐서 연산을 상위층에 넣는 잔차연결 기법도 활용. (1 2 3 4 층이 있을 때 1 -> 2 -> 3 -> 4 로 가는 방향은 동일하나, 2가 3과 4 모두 가는 느낌)

## 7.1.1 함수형 API

In [1]:
from keras import Input, layers

input_tensor = Input(shape=(32,))
dense = layers.Dense(32, activation='relu')

output_tensor = dense(input_tensor)

In [2]:
# 간단한 예시를 통해 Sequential과 함수형 API를 비교

from keras.models import Sequential, Model
from keras import layers
from keras import Input

seq_model = Sequential([
    layers.Dense(32, activation='relu', input_shape=(64,)),
    layers.Dense(32, activation='relu'),
    layers.Dense(10, activation='softmax')
])

input_tensor = Input(shape=(64,))
x = layers.Dense(32, activation='relu')(input_tensor)
x = layers.Dense(32, activation='relu')(x)
output_tensor = layers.Dense(10, activation='softmax')(x)

model = Model(input_tensor, output_tensor)       # 입력텐서와 출력텐서만 가지고 모델 객체 생성
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 64)]              0         
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 32)                1056      
                                                                 
 dense_6 (Dense)             (None, 10)                330       
                                                                 
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


## 7.1.2 다중 입력 모델

In [6]:
# 2개의 입력을 가진 질문-응답 모델의 함수형 API

from keras.models import Model
from keras import layers
from keras import Input

text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

text_input = Input(shape=(None,), dtype='int32', name='text')      # 길이가 정해지지 않은 정수 시퀀스 입력

embedded_text = layers.Embedding(text_vocabulary_size, 64)(text_input)
encoded_text = layers.LSTM(32)(embedded_text)

question_input = Input(shape=(None, ), dtype='int32', name='question')

embedded_question = layers.Embedding(question_vocabulary_size, 32)(question_input)
encoded_question = layers.LSTM(16)(embedded_question)

concatenated = layers.concatenate([encoded_text, encoded_question], axis=-1)    # 인코딩된 질문과 텍스트를 연결

answer = layers.Dense(answer_vocabulary_size, activation='softmax')(concatenated)

model = Model([text_input, question_input], answer)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

## 7.1.3 다중 출력 모델

In [9]:
# 3개의 출력을 가진 함수형 API

from keras import layers
from keras import Input
from keras.models import Model
vocabulary_size = 50000
num_income_groups = 10

post_input = Input(shape=(None,), dtype='int32', name='posts')
embedded_posts = layers.Embedding(vocabulary_size, 256)(post_input)

x = layers.Conv1D(128, 5, activation='relu')(embedded_posts)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation='relu')(x)

age_prediction = layers.Dense(1, name='age')(x)
income_prediction = layers.Dense(num_income_groups, activation='softmax', name='income')(x)
gender_prediction = layers.Dense(1, activation='sigmoid', name='gender')(x)

model = Model(post_input, [age_prediction, income_prediction, gender_prediction])

model.compile(optimizer='rmsprop', loss=['mse', 'categorical_crossentropy', 'binary_crossentropy'])
# 또는 loss={'age':'mse', 'income':'categorical_crossentropy', 'gender':'binary_crossentropy'}

## 7.1.4 층으로 구성된 비순환 유향 그래프

- 비순환 유향 그래프 : 텐서 x가 자기 자신을 출력하는 층의 입력이 될 수 없음

### 인셉션 모듈 : 네트워크 안의 네트워크 구조

In [ ]:
# input shape 없어서 실행하면 에러남

from keras import layers

branch_a = layers.Conv2D(128, 1, activation='relu', strides=2)(x)

branch_b = layers.Conv2D(128, 1, activation='relu')(x)
branch_b = layers.Conv2D(128, 3, activation='relu', strides=2)(branch_b)

branch_c = layers.AveragePooling2D(3, strides=2)(x)
branch_c = layers.Conv2D(128, 3, acitvation='relu')(branch_c)

branch_d = layers.Conv2D(128, 1, activation='relu')(x)
branch_d = layers.Conv2D(128, 3, activation='relu')(branch_d)
branch_d = layers.Conv2D(128, 3, activation='relu', strides=2)(branch_d)

output = layers.concatenate([branch_a, branch_b, branch_c, branch_d], axis=-1)

### 잔차 연결

Gradient Vanishing과 Representational bottleneck을 해결

In [ ]:
from keras import layers

x = ...
y = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
y = layers.Conv2D(128, 3, activation='relu', padding='same')(y)
y = layers.Conv2D(128, 3, activation='relu', padding='same')(y)

y = layers.add([y, x])      # 원본 x를 출력 특성에 더함
# 위에처럼 원본을 넣어도 되고, 합성곱층을 한번 거친 값을 x 대신 넣어줘도 됨

## 7.1.5 층 가중치 공유

In [13]:
from keras import layers
from keras import Input
from keras.models import Model

lstm = layers.LSTM(32)
left_input = Input(shape=(None, 128))
left_output = lstm(left_input)

right_input = Input(shape=(None, 128))
right_output = lstm(right_input)

merged = layers.concatenate([left_output, right_output], axis=-1)
predictions = layers.Dense(1, activation='sigmoid')(merged)

model = Model([left_input, right_input], predictions)

## 7.1.6 층과 모델

함수형 API에서는 모델을 층처럼 이용 가능

In [14]:
from keras import layers
from keras import applications
from keras import Input

xception_base = applications.Xception(weights=None, include_top=False)
left_input = Input(shape=(250, 250, 3))
right_input = Input(shape=(250, 250, 3))

left_features = xception_base(left_input)
right_features = xception_base(right_input)

merged_features = layers.concatenate([left_features, right_features], axis=-1)

# 7.2 케라스 콜백과 텐서보드를 사용한 딥러닝 모델 검사와 모니터링

## 7.2.1 콜백을 사용하여 모델의 훈련 과정 제어하기

- 모델 체크포인트 저장 : 훈련하는 동안 어떤 지점에서 모델의 현재 가중치를 저장
- 조기 종료 : 검증 손실이 더이상 향상되지 않을 때 훈련을 중지
- 훈련하는 동안 하이퍼파라미터 값을 동적으로 조정
- 훈련과 검증 지표를 로그에 기록하거나 시각화

### ModelCheckpoint와 EarlyStopping 콜백

In [ ]:
import keras

callbacks_list = [
    keras.callbacks.EarlyStopping(monitor='val_acc', patience=1),        # 1 에포크보다 더 길게 정확도가 향상되지 않으면 훈련 중지
    keras.callbacks.ModelCheckpoint(filepath='my_model.h5', monitor='val_loss', save_best_only=True)    # 에포크마다 가중치 저장
]

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

model.fit(x, y, epochs=10, batch_size=32, callbacks=callbacks_list, validation_data=(x_val, y_val))

### ReduceLROnPlateau 콜백

검증 손실이 향상되지 않을 때 학습률을 작게 조정

In [ ]:
callbacks_list = [
    keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10)
    # 콜백이 호출될 때 학습률을 factor(10배로 줄임)만큼 조정
]

model.fit(x, y, eopchs=10, batch_size=32, callbacks=callbacks_list, validation_data=(x_val, y_val))

### 나만의 콜백 만들기

In [15]:
import keras
import numpy as np

class ActivationLogger(keras.callbacks.Callback):
    
    def set_model(self, model):
        self.model = model
        layers_outputs = [layer.output for layer in model.layers]
        self.activations_model = keras.models.Model(model.input, layer_outputs)
        
    def on_epoch_end(self, epoch, logs=None):
        if self.validation_data is None:
            raise RuntimeError('Requires validation_data.')
        
        validation_sample = self.validation_Data[0][0:1]
        activations = self.activations_model.predict(validation_sample)
        f = open('activations_at_epoch_' + str(epoch) + '.npz', 'wb')
        np.savez(f, activations)
        f.close()

## 7.2.2 텐서보드

In [16]:
import keras
from keras import layers
from keras.datasets import imdb
from keras.utils import pad_sequences

max_features = 2000
max_len = 500

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

model = keras.models.Sequential([
    layers.Embedding(max_features, 128, input_length=max_len, name='embed'),
    layers.Conv1D(32, 7, activation='relu'),
    layers.MaxPooling1D(5),
    layers.Conv1D(32, 7, activation='relu'),
    layers.GlobalMaxPooling1D(),
    layers.Dense(1)
])

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

In [17]:
callbacks = [keras.callbacks.TensorBoard(log_dir = 'my_log_dir', histogram_freq = 1, embeddings_freq=1)]
  # 1 에포크마다 활성화 출력의 히스토그램을, 임베딩 데이터를 기록
    
history = model.fit(X_train, y_train, epochs=20, batch_size=128, validation_split=0.2, callbacks=callbacks)

Epoch 1/20
157/157 [==============================] - 29s 176ms/step - loss: 0.6192 - acc: 0.6430 - val_loss: 0.5599 - val_acc: 0.7920
Epoch 2/20
157/157 [==============================] - 29s 186ms/step - loss: 0.4487 - acc: 0.8461 - val_loss: 0.4124 - val_acc: 0.8594
Epoch 3/20
157/157 [==============================] - 30s 193ms/step - loss: 0.3798 - acc: 0.8797 - val_loss: 0.3965 - val_acc: 0.8638
Epoch 4/20
157/157 [==============================] - 28s 180ms/step - loss: 0.3178 - acc: 0.9044 - val_loss: 0.5054 - val_acc: 0.8596
Epoch 5/20
157/157 [==============================] - 28s 179ms/step - loss: 0.2996 - acc: 0.9197 - val_loss: 0.6636 - val_acc: 0.8514
Epoch 6/20
157/157 [==============================] - 28s 181ms/step - loss: 0.2396 - acc: 0.9405 - val_loss: 0.6455 - val_acc: 0.8674
Epoch 7/20
157/157 [==============================] - 28s 180ms/step - loss: 0.2139 - acc: 0.9518 - val_loss: 0.6893 - val_acc: 0.8738
Epoch 8/20
157/157 [==============================] - 2

In [25]:
# 실행중에 http://localhost:6006 에 접속하면 나옴
!tensorboard --logdir=my_log_dir

^C


# 7.3 모델의 성능을 최대로 끌어올리기

## 7.3.1 고급 구조 패턴

### 배치 정규화

- 정규화 : 머신러닝 모델에 주입되는 샘플들을 균일하게 만드는 방법(표준정규화)

하지만 네트워크의 각각의 층에서 변환된 후에도 정규화된 데이터가 출력되진 않음 -> 배치정규화

훈련하는동안 평균과 분산이 바뀌더라도 이에 맞게 데이터를 계속 정규화하기 때문에, 그라디언트의 전파에 큰 도움

In [ ]:
# 일반적으로 합성곱이나 완전연결층(Dense) 다음에 사용

conv_model.add(layers.Conv2D(32, 3, activation='relu'))
conv_model.add(layers.BatchNormalization())

conv_model.add(layers.Dense(32, activation='relu'))
conv_model.add(layers.BatchNormalization())

### 깊이별 분리 합성곱

Conv2D를 대체하면서 더 가벼우며 성능이 좋은 층

제한된 데이터로 작은 모델을 처음부터 훈련시킬 때 효과적

In [20]:
from keras.models import Sequential, Model
from keras import layers

height=64
width=64
channels=3
num_classes=10

model = Sequential([
    layers.SeparableConv2D(32, 3, activation='relu', input_shape=(height, width, channels,)),
    layers.SeparableConv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(2),
    
    layers.SeparableConv2D(64, 3, activation='relu'),
    layers.SeparableConv2D(128, 3, activation='relu'),
    layers.MaxPooling2D(2),
    
    layers.SeparableConv2D(64, 3, activation='relu'),
    layers.SeparableConv2D(128, 3, activation='relu'),
    layers.GlobalAveragePooling2D(),
    
    layers.Dense(32, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

## 7.3.2 하이퍼파라미터 최적화

1. 일련의 하이퍼파라미터를 선택
2. 선택된 하이퍼파라미터로 모델 생성
3. 훈련 데이터에 학습하고 검증 데이터에서 최종 성능 측정
4. 다음으로 시도할 하이퍼파라미터를 선택
5. 이 과정 반복
6. 마지막으로 테스트셋에서 성능 측정

## 7.3.3 모델 앙상블

- 앙상블 : 다른 여러개 모델의 예측을 합쳐 더 좋은 예측을 만듬